# Torus one-point functions of the $O(n)$ model

In [34]:
Pkg.activate("."); # activate the parent environment
using BootstrapVirasoro, BootstrapVirasoro.LoopModels, BenchmarkTools
println("Number of threads: $(Threads.nthreads())")

  Activating 

Number of threads: 4


project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`


In [35]:
const Sig = Channels{Rational}

# Spectrum of O(n) model
c = CC(β=sqrt(big"2")/(big"0.8"+big"0.2"*im))
diag_field = Field(c, r=0, s=big"0.4"+big"0.1"*im)
ndiag_indices = [(r, s) for r in 1//2:1//2:20 for s in -1+1//(2r):1//(2r):1 if r * s % 1 == 0]
fields = vcat([Field(c, r = r, s = s) for (r, s) in ndiag_indices], diag_field)

function precompute_blocks(indices, fields; parity, precision=10)
    setprecision(BigFloat, floor(Int, 1.2*precision), base=10)
    Δmax = floor(Int, 0.75*precision)
    co = Correlation(Field(c, r=indices[1], s=indices[2]), Δmax)
    parity != 0 && (fields = filter(V -> V.diagonal || V.s >= 0, fields))
    f = V -> IBlock(co, V, parity=parity)
    spec = ChanSpec(co, fields, f)
    return @channels spec
end

function compute_diagblocks(specs, Ps, chan)
    c = specs.s.corr.fields[1].c
    return [IBlock(specs.s.corr, chan, Field(c, P=P)) for P in Ps]
end

# Solve crossing symmetry for given signature
# Optionnally for many different diagonal blocks in the channel
function solve(
        specs, signature;
        rmax=3, show=true, 
        Ps=nothing, diagchan=nothing,
        fix=nothing
    )
    specs = @channels filter(
        V -> V.diagonal && signature[chan] == 0 || !V.diagonal && V.r >= signature[chan],
        specs[chan]
    )
    if Ps === nothing
        res = solve_bootstrap(specs, fix=fix)
        if show
            Base.show(stdout, res.str_cst, rmax=rmax)
        end
    else
        diagblocks = compute_diagblocks(specs, Ps, diagchan)
        res = solve_bootstrap_manyP(specs, diagblocks)
    end
    return res
end

solve (generic function with 1 method)

## r = 0

In [36]:
ind = (0, sqrt(big"2")*(big"0.4"+big"0.2"*im)) # P generic
blocks = precompute_blocks(ind, fields, parity=1, precision=20)
sol = solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬────────────────────────┬───────────┐
│ Field           │ Structure constant     │ Rel. err. │
├─────────────────┼────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im              │ 0         │
│ (1//2, 0)       │ 0.416666+0.12037im     │ 3.7e-20   │
│ (1, 0)          │ 0.251697-0.0896142im   │ 5.7e-17   │
│ (1, 1)          │ -0.00288987-0.315723im │ 3.4e-17   │
│ (3//2, 0)       │ 0.0631933-0.0402079im  │ 9.9e-12   │
│ (3//2, 2//3)    │ -0.0976787-0.273136im  │ 2e-12     │
│ (2, 0)          │ -0.0325322+0.030433im  │ 2.9e-05   │
│ (2, 1//2)       │ -0.231744+0.00741577im │ 5.9e-06   │
│ (2, 1)          │ -0.134149-0.199108im   │ 6.2e-06   │
└─────────────────┴────────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────────┬───────────┐
│ Field           │ Structure constant     │ Rel. err. │
├─────────────────┼────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+3.15031e-22im      │ 7.7e-21   │
│ (1//2, 

In [37]:
β = c.β
ind = (0, β * (β - 2/β)) # P = P_(1, 2)
blocks = precompute_blocks(ind, fields, parity=1, precision=30)
solve(blocks, Sig(0, 0, 0), rmax=2);

 Channel s
┌─────────────────┬─────────────────────────┬───────────┐
│ Field           │ Structure constant      │ Rel. err. │
├─────────────────┼─────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im               │ 0         │
│ (1//2, 0)       │ 0.355233-0.12359im      │ 5.5e-24   │
│ (1, 0)          │ 0.202314-0.121132im     │ 1.5e-21   │
│ (1, 1)          │ -0.0591978-0.0718335im  │ 1.3e-21   │
│ (3//2, 0)       │ 0.0700839-0.0659127im   │ 1.6e-16   │
│ (3//2, 2//3)    │ -0.144613-0.0452004im   │ 2.7e-17   │
│ (2, 0)          │ 0.0413411+0.000518487im │ 3.1e-10   │
│ (2, 1//2)       │ -0.110954+0.0600867im   │ 5.3e-11   │
│ (2, 1)          │ -0.143968+0.00561456im  │ 4.3e-11   │
└─────────────────┴─────────────────────────┴───────────┘
 Channel t
┌─────────────────┬─────────────────────────┬───────────┐
│ Field           │ Structure constant      │ Rel. err. │
├─────────────────┼─────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+5.20134e-25im       │ 2.2e

## $r = 1, s = 0$

In [38]:
blocks = precompute_blocks((1, 0), fields, parity=1, precision=80);

### Signature $(0, 0, 0)$

In [39]:
using BarnesDoubleGamma
setprecision(BigFloat, 20, base=10)
β = c.β
DG = DoubleGamma(β)
V = Field(c, r=3//2, s=2//3)
V1 = Field(c, r=1, s=0)
V2 = Field(c, r=0, s=β^2)
println(V1[:left].Δ ≈ V2[:left].Δ)
println(LoopModels.Cref(V2, V, V) / LoopModels.Cref(V1, V, V))
println(gamma(1))

true
2.000000000630581425861 + 6.185447576316674431699e-10im
1.0


In [40]:
ϵ = 1e-8
V0 = Field(c, r=0, s=1//2)
V = Field(c, r=1, s=0)
V1 = Field(c, r=1, s=0)
V2 = Field(c, r=0, s=β^2)
println(LoopModels.Cref(V2, V0, V) / LoopModels.Cref(V1, V0, V))
println(LoopModels.Bref(V1, DG))
println(2 / DG(2β)^2 / DG(β + 1 / β)^2)

1.000000000000000000027 + 0.0im
101.1813198816819123621 + 28.95437238763039073156im
101.1801640727629699812 + 28.95462018026876427779im


In [41]:
# show(stdout, sol.str_cst, backend=:latex, rmax=3)

## $r=1, s = 2$

In [42]:
blocks = precompute_blocks((1, 2), fields, parity=0, precision=60);
sol = solve(blocks, Sig(0, 0, 0));

 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ -4.40242+4.03791im         │ 3.7e-26   │
│ (1, 0)          │ -10.1201+11.9725im         │ 8.1e-24   │
│ (1, 1)          │ 7.34646+7.71471im          │ 2.3e-22   │
│ (3//2, 0)       │ -6.96244+12.7403im         │ 1.4e-18   │
│ (3//2, -2//3)   │ 17.0082-3.12202im          │ 2.3e-18   │
│ (3//2, 2//3)    │ 29.3043+3.40064im          │ 3.6e-18   │
│ (2, 0)          │ -10.6338+3.27378im         │ 3.9e-11   │
│ (2, 1//2)       │ 29.2332-26.2693im          │ 1.9e-11   │
│ (2, -1//2)      │ 18.1994-22.7161im          │ 8.7e-12   │
│ (2, 1)          │ 50.421-10.7106im           │ 1.2e-11   │
│ (5//2, 0)       │ -64.0531-13.8044im         │ 0.011     │
│ (5//2, -2//5)   │ -0.147348-30.4599im        │ 0.02      │
│ (5//2, 2//5

In [43]:
# compare with V_{P_{(1, 2)}}
β = c.β
blocks = precompute_blocks((0, β * (β - 2/β)), fields, parity=0, precision=30)
solve(blocks, Sig(0, 0, 0))

 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.355233-0.12359im         │ 8.1e-24   │
│ (1, 0)          │ 0.202314-0.121132im        │ 3.5e-21   │
│ (1, 1)          │ -0.0591978-0.0718335im     │ 4.2e-21   │
│ (3//2, 0)       │ 0.0700839-0.0659127im      │ 2.3e-16   │
│ (3//2, 2//3)    │ -0.144613-0.0452004im      │ 7.7e-17   │
│ (3//2, -2//3)   │ -0.144613-0.0452004im      │ 1e-16     │
│ (2, 0)          │ 0.0413411+0.000518487im    │ 7.8e-10   │
│ (2, -1//2)      │ -0.110954+0.0600867im      │ 4.5e-10   │
│ (2, 1//2)       │ -0.110954+0.0600867im      │ 1.5e-10   │
│ (2, 1)          │ -0.143968+0.00561456im     │ 3.1e-10   │
│ (5//2, 0)       │ 0.0775164-0.0934745im      │ 0.14      │
│ (5//2, -2//5)   │ 0.0998947-0.0360219im      │ 0.12      │
│ (5//2, 2//5

BootstrapSystem{Complex{BigFloat}}
< (P=0.27+-0.35im) >
Number of positions: 58
Matrix size: (116, 110)


## r = 2

In [44]:
blocks = precompute_blocks((2, 0), fields, parity=1, precision=25);

### Signature $(\frac12, \frac12, \frac12)$

In [45]:
solve(blocks, Sig(1//2, 1//2, 1//2), rmax=3//2);

 Channel s
┌──────────────┬────────────────────┬───────────┐
│ Field        │ Structure constant │ Rel. err. │
├──────────────┼────────────────────┼───────────┤
│ (1//2, 0)    │ 1.0+0.0im          │ 0         │
│ (1, 0)       │ -34.9208-45.5559im │ 5.3e-21   │
│ (1, 1)       │ 31.0888+25.0735im  │ 2.6e-21   │
│ (3//2, 0)    │ -589.454+35.4061im │ 2.2e-17   │
│ (3//2, 2//3) │ 277.297-26.5527im  │ 2.6e-17   │
└──────────────┴────────────────────┴───────────┘
 Channel t
┌──────────────┬────────────────────┬───────────┐
│ Field        │ Structure constant │ Rel. err. │
├──────────────┼────────────────────┼───────────┤
│ (1//2, 0)    │ 1.0+1.11975e-22im  │ 2.8e-22   │
│ (1, 0)       │ -34.9208-45.5559im │ 9.2e-22   │
│ (1, 1)       │ 31.0888+25.0735im  │ 1.4e-21   │
│ (3//2, 0)    │ -589.454+35.4061im │ 5.9e-19   │
│ (3//2, 2//3) │ 277.297-26.5527im  │ 2.6e-19   │
└──────────────┴────────────────────┴───────────┘
 Channel u
┌──────────────┬────────────────────┬───────────┐
│ Field        │ 

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = 0$

In [46]:
fix = [(:s, diag_field, 1.0), (:s, Field(c, r=1//2, s=0), 0.0)]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3//2);
println("With $(length(fix)-1) constant fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=1//2);

With 2 constants fixed:
 Channel s
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)       │ 0.0+0.0im          │ 0         │
│ (1, 0)          │ -934.438+210.945im │ 1e-20     │
│ (1, 1)          │ -597.189+296.177im │ 4.2e-21   │
│ (3//2, 0)       │ 1538.45+4136.88im  │ 6.4e-17   │
│ (3//2, 2//3)    │ 1587.23+3847.92im  │ 2.7e-17   │
└─────────────────┴────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0-1.0416e-20im   │ 3.3e-21   │
│ (1//2, 0)       │ 0                  │ 0.43      │
│ (1, 0)          │ -934.438+210.945im │ 3.2e-21   │
│ (1, 1)          │ -597.189+296.177im │ 2.5e-21   │
│ (3//2, 0)       │ 1538.45+4136.88im  │ 2.3e-18   │


We find that the space of solutions has dimension 2.

## r = 3

In [47]:
blocks = precompute_blocks((3, 0), fields, parity=1, precision=20);

### Signature $(1, 1, 1)$

In [48]:
solve(blocks, Sig(1, 1, 1), rmax=2);

 Channel s
┌──────────────┬──────────────────────┬───────────┐
│ Field        │ Structure constant   │ Rel. err. │
├──────────────┼──────────────────────┼───────────┤
│ (1, 0)       │ 1.0+0.0im            │ 0         │
│ (1, 1)       │ -1.12641-0.0491036im │ 7.2e-15   │
│ (3//2, 0)    │ 0                    │ 3.9       │
│ (3//2, 2//3) │ 0                    │ 2.9       │
│ (2, 0)       │ 10728.1+15111.3im    │ 3e-08     │
│ (2, 1//2)    │ -10603.0-14053.7im   │ 1.5e-08   │
│ (2, 1)       │ 10112.1+11149.5im    │ 1.8e-08   │
└──────────────┴──────────────────────┴───────────┘
 Channel t
┌──────────────┬──────────────────────┬───────────┐
│ Field        │ Structure constant   │ Rel. err. │
├──────────────┼──────────────────────┼───────────┤
│ (1, 0)       │ 1.0-4.17794e-15im    │ 4.9e-15   │
│ (1, 1)       │ -1.12641-0.0491036im │ 7.3e-15   │
│ (3//2, 0)    │ 0                    │ 1.4       │
│ (3//2, 2//3) │ 0                    │ 6.6       │
│ (2, 0)       │ 10728.1+15111.3im    │ 1.

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = D^{(s)}_{(1, 0)} = 0$

In [49]:
fix = [
    (:s, diag_field, 1),
    (:s, Field(c, r=1//2, s=0), 0),
    (:s, Field(c, r=1, s=0), 0)
]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3//2);
println("With $(length(fix)-1) constant fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=1);

With 3 constants fixed:
 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 249.309-34.0451im          │ 5.8e-12   │
│ (3//2, 0)       │ -1.17386e+06-5.49204e+06im │ 1.1e-12   │
│ (3//2, 2//3)    │ -1.46418e+06-4.26679e+06im │ 1.1e-12   │
└─────────────────┴────────────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0-3.12744e-13im          │ 2.7e-13   │
│ (1//2, 0)       │ 0                          │ 6.5       │
│ (1, 0)          │ 0                  

The space of solutions has dimension $3 = |\mathcal{M}^c_{1, 1}(3)|$.

## r = 4

In [50]:
blocks = precompute_blocks((4, 0), fields, parity=1, precision=20);

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = D^{(s)}_{(1, 0)} = D^{(s)}_{(1, 1)} = 0$

In [51]:
fix = [
    (:s, diag_field, 1),
    (:s, Field(c, r=1//2, s=0), 0),
    (:s, Field(c, r=1, s=0), 0),
    (:s, Field(c, r=1, s=1), 0)
]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=3//2);
println("With $(length(fix)-1) constant fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=1);

With 4 constants fixed:
 Channel s
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im          │ 0         │
│ (1//2, 0)       │ 0.0+0.0im          │ 0         │
│ (1, 0)          │ 0.0+0.0im          │ 0         │
│ (1, 1)          │ 0.0+0.0im          │ 0         │
│ (3//2, 0)       │ 32440.3+62680.7im  │ 1.3e-07   │
│ (3//2, 2//3)    │ -16345.9-30713.7im │ 2.7e-07   │
└─────────────────┴────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────┬───────────┐
│ Field           │ Structure constant │ Rel. err. │
├─────────────────┼────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+3.10777e-10im  │ 2.1e-10   │
│ (1//2, 0)       │ 0                  │ 1.7       │
│ (1, 0)          │ 0                  │ 1.6       │
│ (1, 1)          │ 0                  │ 1.5       │
│ (3//2, 0)       │ 32440.3+62680.7im  │ 2.4e-08   │


The space of solutions has dimension $4 = |\mathcal{M}^c_{1, 1}(4)|$.

## r = 5

In [52]:
blocks = precompute_blocks((5, 0), fields, parity=0, precision=30);

### Signature $(0, 0, 0)$, $D^{(s)}_{(\frac12, 0)} = D^{(s)}_{(1, 0)} = D^{(s)}_{(1, 1)} = D^{(s)}_{(\frac32, 0)} = 0$


In [53]:
fix = [
    (:s, diag_field, 1),
    (:s, Field(c, r=1//2, s=0), 0),
    (:s, Field(c, r=1, s=0), 0),
    (:s, Field(c, r=1, s=1), 0),
    (:s, Field(c, r=3//2, s=0), 0),
]
println("With $(length(fix)) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=2)
# Check that it does not converge if we fix one less constant:
println("With $(length(fix)-1) constants fixed:")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=2);

With 5 constants fixed:
 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)    │ -4.9684e+07+6.17521e+07im  │ 9.2e-12   │
│ (3//2, -2//3)   │ -4.9684e+07+6.17521e+07im  │ 8.9e-12   │
│ (2, 0)          │ 1.55846e+12-1.09072e+12im  │ 1.7e-11   │
│ (2, -1//2)      │ -1.33301e+12+1.12162e+12im │ 1.5e-11   │
│ (2, 1//2)       │ -1.33301e+12+1.12162e+12im │ 1.2e-11   │
│ (2, 1)          │ 1.14192e+12-1.02862e+12im  │ 2e-11     │
└─────────────────┴────────────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────

This shows that the space of solutions has dimension $5 = |\mathcal{M}^c_{1, 1}(5)|$

## r = 6

In [63]:
blocks = precompute_blocks((6, 0), fields, parity=0, precision=40);
blocks_even = precompute_blocks((6, 0), fields, parity=1, precision=40);
blocks_odd = precompute_blocks((6, 0), fields, parity=-1, precision=40);

### Signature $(0, 0, 0)$, $D^{(s)}_{(1, 0)} = D^{(s)}_{(2, 0)} = D^{(s)}_{(2, \frac12)} = D^{(s)}_{(3, 0)} = D^{(s)}_{(3, \frac23)} = 0$

#### With unfixed parity

In [67]:
fix = vcat(
    [(:s, diag_field, 1)],
    [[(:s, V, 0)] for V in [
        Field(c, r=1 // 2, s=0),
        Field(c, r=1, s=0),
        Field(c, r=1, s=1),
        Field(c, r=3 // 2, s=0),
        Field(c, r=3 // 2, s=2 // 3),
        Field(c, r=2, s=0),
        Field(c, r=2, s=1//2),
    ]]...
)
# println("With $(length(fix[1:end-2])÷3) constants fixed identically in each channel:")
# solve(blocks, Sig(0, 0, 0), fix=fix[1:end-2], rmax=9//2);
println("With $(length(fix[1:end-1])) constants fixed")
solve(blocks, Sig(0, 0, 0), fix=fix[1:end-1], rmax=2);
println("With $(length(fix[1:end])) constants fixed")
solve(blocks, Sig(0, 0, 0), fix=fix, rmax=2);

With 7 constants fixed
 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, -2//3)   │ -1.28632e+10+2.24835e+10im │ 1.1       │
│ (3//2, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (2, 0)          │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)       │ 1.28358e+15+2.82158e+15im  │ 1.1       │
│ (2, -1//2)      │ -2.68867e+15-9.99742e+15im │ 1.1       │
│ (2, 1)          │ 2.21449e+15+7.94185e+15im  │ 1.1       │
└─────────────────┴────────────────────────────┴───────────┘
 Channel t
┌─────────────────┬─────────────────────

Although the convergence is not perfect, we observe that the space of states has dimension $8 = |\mathcal M^c_{1, 1}(6)|$
To make the convergence better we impose the parities.

#### With fixed parity

In [68]:
println("With $(length(fix[1:end-1])) constants fixed, parity=1")
solve(blocks_even, Sig(0, 0, 0), fix=fix[1:end-1], rmax=2);

println("With 1 constant fixed, parity=-1")
solve(blocks_odd, Sig(0, 0, 0), rmax=2);

With 7 constants fixed, parity=1
 Channel s
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼────────────────────────────┼───────────┤
│ (P=0.11+0.06im) │ 1.0+0.0im                  │ 0         │
│ (1//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (1, 0)          │ 0.0+0.0im                  │ 0         │
│ (1, 1)          │ 0.0+0.0im                  │ 0         │
│ (3//2, 0)       │ 0.0+0.0im                  │ 0         │
│ (3//2, 2//3)    │ 0.0+0.0im                  │ 0         │
│ (2, 0)          │ 0.0+0.0im                  │ 0         │
│ (2, 1//2)       │ 4.44148e+14+1.12603e+14im  │ 1.2e-18   │
│ (2, 1)          │ -1.83877e+09-3.93323e+08im │ 6.2e-15   │
└─────────────────┴────────────────────────────┴───────────┘
 Channel t
┌─────────────────┬────────────────────────────┬───────────┐
│ Field           │ Structure constant         │ Rel. err. │
├─────────────────┼───────────

1 of the 8 solutions is odd, 7 are even. The odd and even solutions are the even and odd combinations of the two maps with (1, 2, 3) and (1, 3, 2) loops winding around the three cycles of the torus. At $r=6$ this is the first time there are three different numbers of loops winding around each cycle.